In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import seaborn as sns
sns.set(style="whitegrid")

In [2]:
business_df_merged = pd.read_csv('business_df_merged.csv')

In [3]:
business_df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10721 entries, 0 to 10720
Data columns (total 18 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   stars                              10721 non-null  int64  
 1   business_id                        10721 non-null  object 
 2   name                               10721 non-null  object 
 3   city                               10721 non-null  object 
 4   state                              10721 non-null  object 
 5   postal_code                        10721 non-null  object 
 6   latitude                           10721 non-null  float64
 7   longitude                          10721 non-null  float64
 8   categories                         10721 non-null  object 
 9   review_count                       10721 non-null  int64  
 10  attributes.RestaurantsTakeOut      10721 non-null  bool   
 11  attributes.GoodForKids             10721 non-null  boo

In [4]:
X = business_df_merged.drop({'stars','state','business_id','name','review_count','attributes.RestaurantsTakeOut','attributes.GoodForKids','postal_code_y','categories_y','attributes.RestaurantsDelivery','stars_y'},axis = 1)
y = business_df_merged['stars']

In [5]:
X.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10721 entries, 0 to 10720
Data columns (total 7 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   city                               10721 non-null  object 
 1   postal_code                        10721 non-null  object 
 2   latitude                           10721 non-null  float64
 3   longitude                          10721 non-null  float64
 4   categories                         10721 non-null  object 
 5   attributes.RestaurantsPriceRange2  10721 non-null  int64  
 6   attributes.WiFi                    10721 non-null  object 
dtypes: float64(2), int64(1), object(4)
memory usage: 586.4+ KB


In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state= 42)

In [7]:
X_test.reset_index(drop=True, inplace=True)

In [8]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
for col in X:
    if X[col].dtype == 'object':
        X[col] = le.fit_transform(X[col])

In [9]:
X.describe()

,city,postal_code,latitude,longitude,categories,attributes.RestaurantsPriceRange2,attributes.WiFi
count,10721.000000,10721.000000,10721.000000,10721.000000,10721.000000,10721.000000,10721.000000
mean,13.539782,908.986289,49.249359,-123.067450,21.309486,1.741069,0.368809
std,5.525376,486.192467,0.048763,0.105322,10.421334,0.559118,0.491887
min,0.000000,0.000000,49.080595,-123.349961,0.000000,1.000000,0.000000
25%,13.000000,493.000000,49.222585,-123.131280,12.000000,1.000000,0.000000
50%,17.000000,954.000000,49.263982,-123.107465,24.000000,2.000000,0.000000
75%,17.000000,1332.000000,49.280926,-123.030556,30.000000,2.000000,1.000000
max,18.000000,1737.000000,49.459624,-122.753704,38.000000,4.000000,2.000000


In [10]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X,y,test_size=0.3,random_state= 42)

In [11]:
X1_test.reset_index(drop=True, inplace=True)

In [12]:
# Import MinMaxScaler
from sklearn.preprocessing import MinMaxScaler

# Instantiate MinMaxScaler and use it to rescale X_train and X_test
scaler = MinMaxScaler(feature_range=(0,1))
rescaledX_train = scaler.fit_transform(X1_train)
rescaledX_test = scaler.transform(X1_test)

In [13]:
import pickle

In [14]:
rf_model = pickle.load(open('rf_model.sav', 'rb'))


In [15]:
y_pred = rf_model.predict(rescaledX_test)


In [16]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score


print('Accuracy: %.3f' % accuracy_score(y1_test, y_pred))
cf_mtx = confusion_matrix(y1_test, y_pred)
print("Confusion Matrix:")
print(cf_mtx)

Accuracy: 0.888
Confusion Matrix:
[[1945  126]
 [ 235  911]]


In [17]:
pred_df = pd.DataFrame(y_pred, columns=['Prediction'])


In [18]:
rf_df=pd.concat((X_test, pred_df),axis=1)


In [19]:
rf_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3217 entries, 0 to 3216
Data columns (total 8 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   city                               3217 non-null   object 
 1   postal_code                        3217 non-null   object 
 2   latitude                           3217 non-null   float64
 3   longitude                          3217 non-null   float64
 4   categories                         3217 non-null   object 
 5   attributes.RestaurantsPriceRange2  3217 non-null   int64  
 6   attributes.WiFi                    3217 non-null   object 
 7   Prediction                         3217 non-null   int32  
dtypes: float64(2), int32(1), int64(1), object(4)
memory usage: 188.6+ KB


In [20]:
rf_df.to_csv('rf_df.csv', index=False, header=True)